## TensorFlow 2.2 MobileNet 模型部署

In [1]:
#!pip install tensorflow==2.2.0rc2

In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc2


In [3]:
from tensorflow import keras as keras

keras.backend.clear_session()  # For easy reset of notebook state.

In [4]:
# model_path = "saved_model/mobilenet"
model_path = "mobilenet_dog.h5"

model = keras.models.load_model(model_path)

In [5]:
model.summary()

Model: "mobilenet_dog"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)  

### 创建工作区（Workspace）

In [7]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

twinkle
HCdata-xmas-rg
chinaeast2
88756ff4-4246-499c-8dc5-7df71d1048f9


In [8]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = Model.register(workspace=ws,
                       model_name='tf2-mobilenet',
                       model_path=model_path,
                       model_framework=Model.Framework.TENSORFLOW,  # Framework used to create the model.
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=4))

print('Name:', model.name)
print('Version:', model.version)

Registering model tf2-mobilenet
Name: tf2-mobilenet
Version: 1


### 创建 "Conda File":

用default的Conda和pip源 （文件名：myenv_default.yml）：

In [9]:
from azureml.core.conda_dependencies import CondaDependencies

cd = CondaDependencies.create()
cd.add_pip_package("tensorflow==2.2.0rc2")
cd.add_pip_package("cryptography==2.5")
cd.add_pip_package("pillow")
cd.add_pip_package("azureml-defaults")
cd.add_pip_package("azureml-sdk")
cd.add_pip_package("azureml-widgets")
cd.add_pip_package("wget")

cd.save_to_file(base_directory='deploy/', conda_file_path='myenv_default.yml')

print(cd.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - tensorflow==2.2.0rc2
  - cryptography==2.5
  - pillow
  - azureml-defaults
  - azureml-sdk
  - azureml-widgets
  - wget
channels:
- anaconda
- conda-forge



用国内镜像的Conda和pip源（文件名：myenv.yml）:

pip: ali  conda: sjtug

In [10]:
from azureml.core.conda_dependencies import CondaDependencies

cd_cust = CondaDependencies.create()
cd_cust.add_pip_package("tensorflow==2.2.0rc2")
cd_cust.add_pip_package("cryptography==2.5")
cd_cust.add_pip_package("pillow")
cd_cust.add_pip_package("azureml-defaults")
cd_cust.add_pip_package("azureml-sdk")
cd_cust.add_pip_package("azureml-widgets")
cd_cust.add_pip_package("wget")
cd_cust.set_pip_index_url("--index-url https://mirrors.aliyun.com/pypi/simple/")

cd_cust.remove_channel("anaconda")
cd_cust.remove_channel("conda-forge")
cd_cust.add_channel("https://mirrors.sjtug.sjtu.edu.cn/anaconda/pkgs/main/")
cd_cust.add_channel("https://mirrors.sjtug.sjtu.edu.cn/anaconda/cloud/conda-forge/")

cd_cust.save_to_file(base_directory='deploy/', conda_file_path='myenv.yml')

print(cd_cust.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - tensorflow==2.2.0rc2
  - cryptography==2.5
  - pillow
  - azureml-defaults
  - azureml-sdk
  - azureml-widgets
  - wget
  - --index-url https://mirrors.aliyun.com/pypi/simple/
channels:
- https://mirrors.sjtug.sjtu.edu.cn/anaconda/pkgs/main/
- https://mirrors.sjtug.sjtu.edu.cn/anaconda/cloud/conda-forge/



### Build Environment：

用default的Conda和pip源 "myenv_default"：

In [11]:
from azureml.core.runconfig import CondaDependencies
from azureml.core.environment import Environment
# Create the environment
myenv_default = Environment(name="docker-build-tf-default")
# Enable Docker and reference an image
myenv_default.docker.enabled = True

dockerfile = """
FROM mcr.azk8s.cn/azureml/base:intelmpi2018.3-ubuntu16.04
"""

myenv_default.docker.base_image = None
myenv_default.docker.base_dockerfile = dockerfile
# myenv.python.user_managed_dependencies=True
# myenv.python.interpreter_path = "/opt/miniconda/bin/python"
myenv_default.inferencing_stack_version = "latest"  # This will install the inference specific apt packages.

cdenv_default = CondaDependencies.create()
cdenv_default.add_pip_package("tensorflow==2.2.0rc2")
cdenv_default.add_pip_package("cryptography==2.5")
cdenv_default.add_pip_package("pillow")
cdenv_default.add_pip_package("azureml-defaults")
cdenv_default.add_pip_package("azureml-sdk")
cdenv_default.add_pip_package("azureml-widgets")
cdenv_default.add_pip_package("wget")

print(cdenv_default.serialize_to_string())

myenv_default.python.conda_dependencies=cdenv_default

### Register the env for feature use
myenv_default.register(workspace=ws)

from azureml.core import Image
build = myenv_default.build(workspace=ws)
build.wait_for_completion(show_output=True)

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - tensorflow==2.2.0rc2
  - cryptography==2.5
  - pillow
  - azureml-defaults
  - azureml-sdk
  - azureml-widgets
  - wget
channels:
- anaconda
- conda-forge

Image Build Status: Running

2020/04/28 06:56:34 Downloading source code...
2020/04/28 06:56:35 Finished downloading source code
2020/04/28 06:56:35 Creating Docker network: acb_default_network, driver: 'bridge'
2020/04/28 06:56:36 Successfully set up Docker network: acb_default_network
2020/04/28 06:56:36 Setting up Docker configuration...
2020/04/28 06:56:36 Successfully set up Docker

Hit:3 http://security.ubuntu.com/ubuntu xenial-security InRelease
Hit:4 http://ppa.launchpad.net/adiscon/v8-stable/ubuntu xenial InRelease
Hit:5 http://archive.ubuntu.com/ubuntu xenial-backports InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
libunwind8 is already the newest version (1.1-4.1).
unzip is already the newest version (6.0-20ubuntu1).
liblttng-ust0 is already the newest version (2.7.1-1).
libxml++2.6-2v5 is already the newest version (2.40.1-1).
runit is already the newest version (2.1.2-3ubuntu1).
libcurl3 is already the newest version (7.47.0-1ubuntu2.14).
psmisc is already the newest version (22.21-2.1ubuntu0.1).
wget is already the newest version (1.17.1-1ubuntu1.5).
The following packages were automatically installed and are no longer required:
  dh-python distro-info-data file gir1.2-glib-2.0 iso-codes libapt-inst2.0
  libdbus-glib-1-2 libgirepository-1.0-1 libmagic1 libmpdec2 libpython3-stdlib
  libp


certifi-2020.4.5.1   | 159 KB    |            |   0% 
certifi-2020.4.5.1   | 159 KB    | 7          |   8% 
certifi-2020.4.5.1   | 159 KB    | ########## | 100% 

ncurses-6.0          | 907 KB    |            |   0% 
ncurses-6.0          | 907 KB    | 1          |   1% 
ncurses-6.0          | 907 KB    | 3          |   4% 
ncurses-6.0          | 907 KB    | ##7        |  28% 
ncurses-6.0          | 907 KB    | #######9   |  79% 
ncurses-6.0          | 907 KB    | ########9  |  90% 
ncurses-6.0          | 907 KB    | ########## | 100% 

libffi-3.2.1         | 43 KB     |            |   0% 
libffi-3.2.1         | 43 KB     | ##8        |  28% 
libffi-3.2.1         | 43 KB     | ########## | 100% 

zlib-1.2.11          | 120 KB    |            |   0% 
zlib-1.2.11          | 120 KB    | #          |  10% 
zlib-1.2.11          | 120 KB    | #######    |  70% 
zlib-1.2.11          | 120 KB    | ########## | 100% 

pip-20.0.2           | 1.9 MB    |            |   0% 
pip-20.0.2           | 

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=1330519003715ba3e980a764c313eb311ee8de920584e8a4400082d61d12a9d3
  Stored in directory: /root/.cache/pip/wheels/90/1d/93/c863ee832230df5cfc25ca497b3e88e0ee3ea9e44adc46ac62
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=beacb6632b3408f4a09400c44b63f789fb68bfdc957bf6b1e27718a2065e70d2
  Stored in directory: /root/.cache/pip/wheels/93/2a/eb/e58dbcbc963549ee4f065ff80a59f274cc7210b6eab962acdc
  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none-any.whl size=121931 sha256=a93ed7f380ed095ed9327e238b6103cb135bc9be5a6308acb1d83edd6c076660
  Stored in directory: /root/.cache/pip/wheels/c3/af/84/3962a6af7b4ab336e951b7877dcfb758cf94548bb1771e0679
  Created wheel for wrapt: filename=wrapt-1.12.1-cp36-cp36m-linux_x86_64.whl size=66298 sha256=9da302def6600e35bdf2b86844fba78c675d78be17ac20785800c2539c8ac46c
  Stored in directory: /root/.cache/pip/wheels/32/42/7f/23cae9ff6ef6

#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_7d965178715158edb49968e33513a9ce
#
# To deactivate an active environment, use:
# > source deactivate
#



Removing intermediate container 81860d28b817
 ---> 3a60737870b0
Step 14/21 : ENV PATH /azureml-envs/azureml_7d965178715158edb49968e33513a9ce/bin:$PATH
 ---> Running in 530591d482e1
Removing intermediate container 530591d482e1
 ---> bc6bdff36cb8
Step 15/21 : ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/azureml_7d965178715158edb49968e33513a9ce
 ---> Running in 127e8fcb1da0
Removing intermediate container 127e8fcb1da0
 ---> 6fc7c53b709e
Step 16/21 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_7d965178715158edb49968e33513a9ce/lib:$LD_LIBRARY_PATH
 ---> Running in d05e7f024ff7
Removing intermediate container d05e7f024ff7
 ---> 5a18c9540635
Step 17/21 : COPY azureml-environment-setup/spark_cache.py azureml-environment-setup/log4j.properties /azureml-environment-setup/
 ---> 7505cfac6630
Step 18/21 : RUN if

用国内镜像的Conda和pip源:

In [12]:
from azureml.core.runconfig import CondaDependencies
from azureml.core.environment import Environment
# Create the environment
myenv = Environment(name="docker-build-tf-cust")
# Enable Docker and reference an image
myenv.docker.enabled = True

dockerfile = """
FROM mcr.azk8s.cn/azureml/base:intelmpi2018.3-ubuntu16.04
"""

myenv.docker.base_image = None
myenv.docker.base_dockerfile = dockerfile
# myenv.python.user_managed_dependencies=True
# myenv.python.interpreter_path = "/opt/miniconda/bin/python"
myenv.inferencing_stack_version = "latest"  # This will install the inference specific apt packages.

cdenv = CondaDependencies.create()
cdenv.add_pip_package("tensorflow==2.2.0rc2")
cdenv.add_pip_package("cryptography==2.5")
cdenv.add_pip_package("pillow")
cdenv.add_pip_package("azureml-defaults")
cdenv.add_pip_package("azureml-sdk")
cdenv.add_pip_package("azureml-widgets")
cdenv.add_pip_package("wget")
cdenv.set_pip_index_url("--index-url https://mirrors.aliyun.com/pypi/simple/")

cdenv.remove_channel("anaconda")
cdenv.remove_channel("conda-forge")
cdenv.add_channel("https://mirrors.sjtug.sjtu.edu.cn/anaconda/pkgs/main/")
cdenv.add_channel("https://mirrors.sjtug.sjtu.edu.cn/anaconda/cloud/conda-forge/")

print(cdenv.serialize_to_string())

myenv.python.conda_dependencies=cdenv

### Register the env for feature use
myenv.register(workspace=ws)

from azureml.core import Image
build = myenv.build(workspace=ws)
build.wait_for_completion(show_output=True)

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - tensorflow==2.2.0rc2
  - cryptography==2.5
  - pillow
  - azureml-defaults
  - azureml-sdk
  - azureml-widgets
  - wget
  - --index-url https://mirrors.aliyun.com/pypi/simple/
channels:
- https://mirrors.sjtug.sjtu.edu.cn/anaconda/pkgs/main/
- https://mirrors.sjtug.sjtu.edu.cn/anaconda/cloud/conda-forge/

Image Build Status: Running

2020/04/28 07:08:39 Downloading source code...
2020/04/28 07:08:40 Finished downloading source code
2020/04/28 07:08:41 Creating Docker network: acb_default_network, driver: 'bridge'
2020/04/28 07:08:41 Succes

Reading package lists...
Building dependency tree...
Reading state information...
libunwind8 is already the newest version (1.1-4.1).
unzip is already the newest version (6.0-20ubuntu1).
liblttng-ust0 is already the newest version (2.7.1-1).
libxml++2.6-2v5 is already the newest version (2.40.1-1).
runit is already the newest version (2.1.2-3ubuntu1).
libcurl3 is already the newest version (7.47.0-1ubuntu2.14).
psmisc is already the newest version (22.21-2.1ubuntu0.1).
wget is already the newest version (1.17.1-1ubuntu1.5).
The following packages were automatically installed and are no longer required:
  dh-python distro-info-data file gir1.2-glib-2.0 iso-codes libapt-inst2.0
  libdbus-glib-1-2 libgirepository-1.0-1 libmagic1 libmpdec2 libpython3-stdlib
  libpython3.5-minimal libpython3.5-stdlib lsb-release mime-support
  powermgmt-base python-apt-common python3 python3-apt python3-dbus python3-gi
  python3-minimal python3-pycurl python3-software-properties python3.5
  python3.5-minima

Executing transaction: ...working... done
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/



  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=d22779b7d95e0ff280f0d0d07236d81626d85690340a6ba2be878d84d20e7658
  Stored in directory: /root/.cache/pip/wheels/0f/5c/66/f87adeb91df636dd9dc511a2ae87966e8ca31eb0ead5ecea6c
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=46531ebcf8ed47b37b032c662110ac6f4aa10dd6628ad7f0aa960ae3e944d171
  Stored in directory: /root/.cache/pip/wheels/8b/49/fb/8c8dfa26e09f80c498130cf95189975bcc8735c4feae69f3c1
  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none-any.whl size=121931 sha256=c0f6723d488de2eddaecb12c58b0e811a8fcd970626c71e010f6a9436a4ddf24
  Stored in directory: /root/.cache/pip/wheels/93/57/cf/9f2f468a53488998359931532f958b0579dbca51ba2b00b90d
  Created wheel for wrapt: filename=wrapt-1.12.1-cp36-cp36m-linux_x86_64.whl size=66283 sha256=4abadc510afcecaeadc4a8d67ab83c250eae5a043f6dc132a426f96a1f2e61eb
  Stored in directory: /root/.cache/pip/wheels/9e/47/e8/a7db270d08cf

#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_9618d426b5044c2f95ccb09247014d72
#
# To deactivate an active environment, use:
# > source deactivate
#



Removing intermediate container a741d4559a67
 ---> 6ca2a3c06641
Step 14/21 : ENV PATH /azureml-envs/azureml_9618d426b5044c2f95ccb09247014d72/bin:$PATH
 ---> Running in f90b70f3fed6
Removing intermediate container f90b70f3fed6
 ---> 82f334d05f6f
Step 15/21 : ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/azureml_9618d426b5044c2f95ccb09247014d72
 ---> Running in 544f9258f0b1
Removing intermediate container 544f9258f0b1
 ---> bd2b847a4972
Step 16/21 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_9618d426b5044c2f95ccb09247014d72/lib:$LD_LIBRARY_PATH
 ---> Running in 9b7ba70a456b
Removing intermediate container 9b7ba70a456b
 ---> 576cec3691a5
Step 17/21 : COPY azureml-environment-setup/spark_cache.py azureml-environment-setup/log4j.properties /azureml-environment-setup/
 ---> 389f1bd029bf
Step 18/21 : RUN if

### Deploy to ACI (Azure Container Instance)

#### 国内源 基于之前的environment来deploy：

In [13]:
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core import Webservice
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException

myenv=Environment.get(ws, "docker-build-tf-cust")

service_name = 'my-custom-env-service'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

inference_config_cust = InferenceConfig(entry_script="deploy/score.py",
                                        environment = myenv)
aci_config_cust = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                     memory_gb=1)

cust_service = Model.deploy(workspace=ws, 
                            name=service_name, 
                            models=[model], 
                            inference_config=inference_config_cust, 
                            deployment_config=aci_config_cust)


cust_service.wait_for_deployment(show_output = True)
print(cust_service.state)
print(cust_service.get_logs())

webservice_cust = ws.webservices[service_name]
print("Webservice: {}, scoring URI: {}".format(service_name, webservice_cust.scoring_uri))

Running.........................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
/usr/sbin/nginx: /azureml-envs/azureml_9618d426b5044c2f95ccb09247014d72/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9618d426b5044c2f95ccb09247014d72/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9618d426b5044c2f95ccb09247014d72/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9618d426b5044c2f95ccb09247014d72/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9618d426b5044c2f95ccb09247014d72/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-04-28T08:06:37,618629255+00:00 - rsyslog/run 
2020-04-28T08:06:

#### default源 基于之前的environment来deploy：

In [14]:
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core import Webservice
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException

myenv_default=Environment.get(ws, "docker-build-tf-default")

service_name = 'my-env-service'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

inference_config = InferenceConfig(entry_script="deploy/score.py",
                                   environment = myenv_default)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                memory_gb=1)

service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aci_config)


service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

webservice = ws.webservices[service_name]
print("Webservice: {}, scoring URI: {}".format(service_name, webservice.scoring_uri))

Running...................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
2020-04-28T08:13:06,611758789+00:00 - gunicorn/run 
2020-04-28T08:13:06,614382227+00:00 - rsyslog/run 
2020-04-28T08:13:06,619219296+00:00 - iot-server/run 
2020-04-28T08:13:06,633053893+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_7d965178715158edb49968e33513a9ce/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7d965178715158edb49968e33513a9ce/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7d965178715158edb49968e33513a9ce/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7d965178715158edb49968e33513a9ce/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azurem

#### 基于conda file （.yml文件）：

In [15]:
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core import Webservice
from azureml.core.webservice import AciWebservice
from azureml.exceptions import WebserviceException

service_name = 'my-conda-file-service'

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

inference_config_cf = InferenceConfig(runtime= "python", 
                                      entry_script="deploy/score.py",
                                      conda_file="deploy/myenv.yml")
aci_config_cf = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                memory_gb=1)

service_cf = Model.deploy(workspace=ws, 
                          name=service_name, 
                          models=[model], 
                          inference_config=inference_config_cf, 
                          deployment_config=aci_config_cf)


service_cf.wait_for_deployment(show_output = True)
print(service_cf.state)
print(service_cf.get_logs())

webservice_cf = ws.webservices[service_name]
print("Webservice: {}, scoring URI: {}".format(service_name, webservice_cf.scoring_uri))

Running.......................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
2020-04-28T08:25:18,565357577+00:00 - gunicorn/run 
2020-04-28T08:25:18,563273990+00:00 - iot-server/run 
/usr/sbin/nginx: /azureml-envs/azureml_9618d426b5044c2f95ccb09247014d72/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9618d426b5044c2f95ccb09247014d72/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9618d426b5044c2f95ccb09247014d72/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9618d426b5044c2f95ccb09247014d72/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9

### Test Request:

#### 国内源 Env：

In [16]:
import requests
import json

data = json.dumps({'data': "http://www.mgous.com/img/p/1905/28144928349/1_620_462.jpg"})
data = bytes(data, encoding='utf8')

headers = {'Content-Type':'application/json'}

resp = requests.post(webservice_cust.scoring_uri, data=data, headers=headers)

print("POST to url", webservice_cust.scoring_uri)
print(resp.text)

POST to url http://3da030b4-d4c9-4bea-a05e-a9ae70951cbc.chinaeast2.azurecontainer.console.azure.cn/score
"{\"result\": [[1.7051189842080014e-36, 1.0]]}"


#### Default源 Env：

In [17]:
import requests
import json

data = json.dumps({'data': "http://www.mgous.com/img/p/1905/28144928349/1_620_462.jpg"})
data = bytes(data, encoding='utf8')

headers = {'Content-Type':'application/json'}

resp = requests.post(webservice.scoring_uri, data=data, headers=headers)

print("POST to url", webservice.scoring_uri)
print(resp.text)

POST to url http://de91e3b1-8e9e-470a-9d56-230092ff438e.chinaeast2.azurecontainer.console.azure.cn/score
"{\"result\": [[1.7051189842080014e-36, 1.0]]}"


#### Conda File：

In [18]:
import requests
import json

data = json.dumps({'data': "http://www.mgous.com/img/p/1905/28144928349/1_620_462.jpg"})
data = bytes(data, encoding='utf8')

headers = {'Content-Type':'application/json'}

resp = requests.post(webservice_cf.scoring_uri, data=data, headers=headers)

print("POST to url", webservice_cf.scoring_uri)
print(resp.text)

POST to url http://f8a97acd-a586-44c4-b792-db9b8d08570d.chinaeast2.azurecontainer.console.azure.cn/score
"{\"result\": [[1.7051189842080014e-36, 1.0]]}"


In [11]:
service.delete()